In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import os

import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib

# Data treatment
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split
import pickle

# Base classifiers
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss, accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree

# Multilabel classifiers - Problem Transformation
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.ensemble import RakelD

# Multilabel classifiers - Algorithm Adaptation
from skmultilearn.adapt import BRkNNaClassifier
from skmultilearn.adapt import MLkNN
from skmultilearn.adapt import MLTSVM

# Metrics
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Embedding classifiers
#from skmultilearn.embedding import OpenNetworkEmbedder, CLEMS, SKLearnEmbedder
#from sklearn.manifold import SpectralEmbedding
#from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
#from skmultilearn.embedding import EmbeddingClassifier
#from sklearn.ensemble import RandomForestRegressor

from utils import KunischMetrics
from utils import KunischPruner
from utils import DataExplorer
from utils import KunischPlotter

In [2]:
def build_model(mlb_estimator, xtrain, ytrain, xtest, ytest, model=None):
    """Builds a multilabel estimator and runs it over a given train and test data,
       with an optional base classifier model.

    Parameters:
    mlb_estimator (mlb classifier): a PROBLEM_TRANSFORMATION or ALGORITHM_ADAPTATION 
                                    method from sklearn-multilabel
    xtrain, ytrain, xtest, ytest (np arrays): train and test data
    model (Base Estimator): optional, ignored if mlb_estimator is part of 
                            ALGORITHM_ADAPTATION methods. Base classifier to be 
                            used with the PROBLEM_TRANSFORMATION methods.

    Returns:
    (dict, np.array): dict with metrics (exact match, hamming loss and score) 
                      and array of predictions.
    """
    xtrain = sparse.csr_matrix(xtrain.values)
    ytrain = sparse.csr_matrix(ytrain.values)
    xtest = sparse.csr_matrix(xtest.values)
    ytest = sparse.csr_matrix(ytest.values)
    if model:
      clf = mlb_estimator(model)
    else:
      clf = mlb_estimator
    clf.fit(xtrain, ytrain)
    clf_predictions = clf.predict(xtest)
    return clf_predictions

In [3]:
TRANSF_METHODS = {"BR": BinaryRelevance, "LP": LabelPowerset,
                  "CC": ClassifierChain, "RakelD": RakelD}
mlknn = MLkNN(k=1, s=1)
mltsvm = MLTSVM(c_k=4)
brknna = BRkNNaClassifier(k=1)
ADAPT_METHODS = {"BRkNN": brknna, "MLkNN": mlknn, "MLTSVM": mltsvm}
BASE_CLASSIFIERS = {"LR": LogisticRegression(solver='lbfgs'),
                    "SVC": svm.SVC(), "DT": tree.DecisionTreeClassifier(), "GNB": GaussianNB()}

In [4]:
train_filename = 'augmented_train_df.json'
val_filename = 'val_df.json'
test_filename = 'test_df.json'
K=1

In [5]:
import random
#LABELS_IN_STUDY = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24,
#            26, 30, 34, 38, 40, 50, 60, 80, 100]
LABELS_IN_STUDY = [26]
#for i in range(26, 300, 10):
#    LABELS_IN_STUDY.append(i)

exp_exact_match = {}
exp_hscore = {}
exp_hloss = {}
exp_f1 = {}
exp_precision = {}
exp_recall = {}
exp_acc = {}
exp_1mr = {}
exp_2mr = {}
exp_3mr = {}
exp_4mr = {}
exp_5mr = {}

for meth_name in TRANSF_METHODS.keys():
  for base_name in BASE_CLASSIFIERS.keys():
    exp_exact_match[meth_name + "_" + base_name] = []
    exp_hscore[meth_name + "_" + base_name] = []
    exp_hloss[meth_name + "_" + base_name] = []
    exp_f1[meth_name + "_" + base_name] = []
    exp_precision[meth_name + "_" + base_name] = []
    exp_recall[meth_name + "_" + base_name] = []
    exp_acc[meth_name + "_" + base_name] = []
    exp_1mr[meth_name + "_" + base_name] = []
    exp_2mr[meth_name + "_" + base_name] = []
    exp_3mr[meth_name + "_" + base_name] = []
    exp_4mr[meth_name + "_" + base_name] = []
    exp_5mr[meth_name + "_" + base_name] = []

for meth_name in ADAPT_METHODS.keys():
  exp_exact_match[meth_name] = []
  exp_hscore[meth_name] = []
  exp_hloss[meth_name] = []
  exp_f1[meth_name] = []
  exp_precision[meth_name] = []
  exp_recall[meth_name] = []
  exp_acc[meth_name] = []
  exp_1mr[meth_name] = []
  exp_2mr[meth_name] = []
  exp_3mr[meth_name] = []
  exp_4mr[meth_name] = []
  exp_5mr[meth_name] = []

PREVIOUS_LABELS = 0
USED_FREQS = []

output_dir = os.path.join('outputs', 'base-rn18')
os.makedirs(output_dir, exist_ok = True)

for i in LABELS_IN_STUDY:
  pruner = KunischPruner(i)
  print("Comenzando con i={}".format(i))
    
  # Carga o generacion de top labels
  top_labels = None
  if not os.path.isfile(os.path.join('labels', f'top_{i}L.pickle')):
        save = input(f"Se creará un archivo nuevo para {i} labels con el fold 0. Desea continuar? (y/n)")
        if save == "y":
            labels_dir = os.path.join('labels')
            train_labels = pd.read_json(os.path.join(labels_dir, 'augmented_train_df.json'), orient='index')
            top_labels = pruner.filter_labels(train_labels)
            with open(os.path.join('labels', f'top_{i}L.pickle'), 'wb') as f:
                pickle.dump(top_labels, f)
            print("Top labels creado con éxito")
        else:
            raise Exception("No se logró cargar top_labels")
  else:
        print(f"Usando top_labels previamente generados para {i} labels")
        with open(os.path.join('labels', f'top_{i}L.pickle'), 'rb') as f:
            top_labels = pickle.load(f)
        print(f"top labels previamente generado contiene {len(top_labels)} etiquetas")
  
  pruner.set_top_labels(top_labels)
        
  if len(top_labels) == PREVIOUS_LABELS:
        print(f"Al intentar usar {i} labels, se repitió el valor previo {PREVIOUS_LABELS}. Saltando iteración.")
        continue
        
  PREVIOUS_LABELS = len(top_labels)
  USED_FREQS.append(i)
  

  for meth_name, method in ADAPT_METHODS.items():
        print("-Probando suerte con", meth_name)

        sum_f1 = 0
        sum_recall = 0
        sum_precision = 0
        sum_acc = 0
        sum_hl = 0
        sum_emr = 0
        sum_hs = 0
        
        sum_mr1 = 0
        sum_mr2 = 0
        sum_mr3 = 0
        sum_mr4 = 0
        sum_mr5 = 0


        features_dir = os.path.join('features', 'rn18')
        labels_dir = os.path.join('labels')
        
        features_train = pd.read_json(os.path.join(features_dir, train_filename), orient='index')
        features_val = pd.read_json(os.path.join(features_dir, val_filename), orient='index')
        features_test = pd.read_json(os.path.join(features_dir, test_filename), orient='index')

        labels_train = pd.read_json(os.path.join(labels_dir, train_filename), orient='index')
        labels_val = pd.read_json(os.path.join(labels_dir, val_filename), orient='index')
        labels_test = pd.read_json(os.path.join(labels_dir, test_filename), orient='index')

                
        #shuffled = labels_train.index.values
        #random.shuffle(shuffled)
        
        #labels_train.set_index(pd.Index(shuffled))
        
        #shuffled_test = labels_test.index.values
        #random.shuffle(shuffled_test)
        #labels_test.set_index(pd.Index(shuffled_test))

        # Dataset creation
        X_train = features_train
        X_test = features_test

        Y_train = pruner.filter_df(labels_train) # reduce labels to most freq
        Y_test = pruner.filter_df(labels_test) # in both train and test
        
        with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
            display(X_train.sort_index().head(10))
            display(X_test.sort_index().head(10))
            display(Y_train.sort_index().head(10))
            display(Y_test.sort_index().head(10))
    
        predictions_i = build_model(method, X_train, Y_train, X_test, Y_test)
        metrics = KunischMetrics(Y_test.values, predictions_i)

        micro_f1 = metrics.f1(average='micro')
        micro_recall = metrics.recall(average='micro')
        micro_precision = metrics.precision(average='micro')
        acc = metrics.acc()
        hl = metrics.hl()
        emr = metrics.emr()
        hs = metrics.hs()

        mr1 = metrics.mr1()
        mr2 = metrics.mr2()
        mr3 = metrics.mr3()
        mr4 = metrics.mr4()
        mr5 = metrics.mr5()

        #print("---Micro F1:", micro_f1)
        #print("---Micro recall:", micro_recall)
        #print("---Micro precision:", micro_precision)
        #print("---Accuracy:", acc)
        #print("---Hamming Loss:", hl)
        #print("---Exact Match Ratio:", emr)
        #print("---Hamming Score:", hs)
        #print("---5-Match Ratio:", mr5)

        sum_f1 += micro_f1
        sum_recall += micro_recall
        sum_precision += micro_precision
        sum_acc += acc
        sum_hl += hl 
        sum_emr += emr
        sum_hs += hs

        sum_mr1 += mr1
        sum_mr2 += mr2
        sum_mr3 += mr3
        sum_mr4 += mr4
        sum_mr5 += mr5
        #print("")

        avg_f1 = round(sum_f1/K, 4)
        avg_recall = round(sum_recall/K, 4)
        avg_precision = round(sum_precision/K, 4)
        avg_acc = round(sum_acc/K, 4)
        avg_hl = round(sum_hl/K, 4)
        avg_emr = round(sum_emr/K, 4)
        avg_hs = round(sum_hs/K, 4)
        
        avg_mr1 = round(sum_mr1/K, 4)
        avg_mr2 = round(sum_mr2/K, 4)
        avg_mr3 = round(sum_mr3/K, 4)
        avg_mr4 = round(sum_mr4/K, 4)
        avg_mr5 = round(sum_mr5/K, 4)
        
        print("---AVG Micro F1:", avg_f1)
        print("---AVG Micro recall:", avg_recall)
        print("---AVG Micro precision:", avg_precision)
        print("---AVG Accuracy:", avg_acc)
        print("---AVG Hamming Loss:", avg_hl)
        print("---AVG Exact Match Ratio:", avg_emr)
        print("---AVG Hamming Score:", avg_hs)
        print("---AVG 1-Match Ratio:", avg_mr1)
        print("---AVG 2-Match Ratio:", avg_mr2)
        print("---AVG 3-Match Ratio:", avg_mr3)
        print("---AVG 4-Match Ratio:", avg_mr4)
        print("---AVG 5-Match Ratio:", avg_mr5)
            
        exp_exact_match[meth_name].append(avg_emr)
        exp_hscore[meth_name].append(avg_hs)
        exp_hloss[meth_name].append(avg_hl)
        exp_precision[meth_name].append(avg_precision)
        exp_recall[meth_name].append(avg_recall)
        exp_f1[meth_name].append(avg_f1)
        exp_acc[meth_name].append(avg_acc)
        
        exp_1mr[meth_name].append(avg_mr1)
        exp_2mr[meth_name].append(avg_mr2)
        exp_3mr[meth_name].append(avg_mr3)
        exp_4mr[meth_name].append(avg_mr4)
        exp_5mr[meth_name].append(avg_mr5)
        
        print("")
 
aa = """  
# Linear regression and SVC will raise error if Y_train is composed by only one class
  for meth_name, method in TRANSF_METHODS.items():
        for base_name, classifier in BASE_CLASSIFIERS.items():
            print("-Probando suerte con", meth_name, base_name)

            sum_f1 = 0
            sum_recall = 0
            sum_precision = 0
            sum_acc = 0
            sum_hl = 0
            sum_emr = 0
            sum_hs = 0
            
            sum_mr1 = 0
            sum_mr2 = 0
            sum_mr3 = 0
            sum_mr4 = 0
            sum_mr5 = 0


            features_dir =  os.path.join('features', 'rn18')
            labels_dir = os.path.join('labels')
            features_train = pd.read_json(os.path.join(features_dir, train_filename), orient='index')
            features_val = pd.read_json(os.path.join(features_dir, val_filename), orient='index')
            features_test = pd.read_json(os.path.join(features_dir, test_filename), orient='index')
            labels_train = pd.read_json(os.path.join(labels_dir, train_filename), orient='index')
            labels_val = pd.read_json(os.path.join(labels_dir, val_filename), orient='index')
            labels_test = pd.read_json(os.path.join(labels_dir, test_filename), orient='index')

            
            shuffled = labels_train.index.values
            random.shuffle(shuffled)

            labels_train.set_index(pd.Index(shuffled))
            display(labels_train)

            shuffled_test = labels_test.index.values
            random.shuffle(shuffled_test)
            labels_test.set_index(pd.Index(shuffled_test))

            
            # Dataset creation
            X_train = features_train
            X_test = features_test 

            Y_train = pruner.filter_df(labels_train) # reduce labels to most freq
            Y_test = pruner.filter_df(labels_test) # in both train and test

    
            predictions_i = build_model(method, X_train, Y_train, X_test, Y_test, model=classifier)
            metrics = KunischMetrics(Y_test.values, predictions_i)

            micro_f1 = metrics.f1(average='micro')
            micro_recall = metrics.recall(average='micro')
            micro_precision = metrics.precision(average='micro')
            acc = metrics.acc()
            hl = metrics.hl()
            emr = metrics.emr()
            hs = metrics.hs()

            mr1 = metrics.mr1()
            mr2 = metrics.mr2()
            mr3 = metrics.mr3()
            mr4 = metrics.mr4()
            mr5 = metrics.mr5()
            #print("---Micro F1:", micro_f1)
            #print("---Micro recall:", micro_recall)
            #print("---Micro precision:", micro_precision)
            #print("---Accuracy:", acc)
            #print("---Hamming Loss:", hl)
            #print("---Exact Match Ratio:", emr)
            #print("---Hamming Score:", hs)
            #print("---5-Match Ratio:", mr5)

            sum_f1 += micro_f1
            sum_recall += micro_recall
            sum_precision += micro_precision
            sum_acc += acc
            sum_hl += hl 
            sum_emr += emr
            sum_hs += hs
                
            sum_mr1 += mr1
            sum_mr2 += mr2
            sum_mr3 += mr3
            sum_mr4 += mr4
            sum_mr5 += mr5
            #print("")

            avg_f1 = round(sum_f1/K, 4)
            avg_recall = round(sum_recall/K, 4)
            avg_precision = round(sum_precision/K, 4)
            avg_acc = round(sum_acc/K, 4)
            avg_hl = round(sum_hl/K, 4)
            avg_emr = round(sum_emr/K, 4)
            avg_hs = round(sum_hs/K, 4)
            
            avg_mr1 = round(sum_mr1/K, 4)
            avg_mr2 = round(sum_mr2/K, 4)
            avg_mr3 = round(sum_mr3/K, 4)
            avg_mr4 = round(sum_mr4/K, 4)
            avg_mr5 = round(sum_mr5/K, 4)

            print("---AVG Micro F1:", avg_f1)
            print("---AVG Micro recall:", avg_recall)
            print("---AVG Micro precision:", avg_precision)
            print("---AVG Accuracy:", avg_acc)
            print("---AVG Hamming Loss:", avg_hl)
            print("---AVG Exact Match Ratio:", avg_emr)
            print("---AVG Hamming Score:", avg_hs)
            print("---AVG 1-Match Ratio:", avg_mr1)
            print("---AVG 2-Match Ratio:", avg_mr2)
            print("---AVG 3-Match Ratio:", avg_mr3)
            print("---AVG 4-Match Ratio:", avg_mr4)
            print("---AVG 5-Match Ratio:", avg_mr5)

            exp_exact_match[meth_name + "_" + base_name].append(avg_emr)
            exp_hscore[meth_name + "_" + base_name].append(avg_hs)
            exp_hloss[meth_name + "_" + base_name].append(avg_hl)
            exp_precision[meth_name + "_" + base_name].append(avg_precision)
            exp_recall[meth_name + "_" + base_name].append(avg_recall)
            exp_f1[meth_name + "_" + base_name].append(avg_f1)
            exp_acc[meth_name + "_" + base_name].append(avg_acc)
            exp_1mr[meth_name + "_" + base_name].append(avg_mr1)
            exp_2mr[meth_name + "_" + base_name].append(avg_mr2)
            exp_3mr[meth_name + "_" + base_name].append(avg_mr3)
            exp_4mr[meth_name + "_" + base_name].append(avg_mr4)
            exp_5mr[meth_name + "_" + base_name].append(avg_mr5)
            print("")
"""


Comenzando con i=26
Usando top_labels previamente generados para 26 labels
top labels previamente generado contiene 26 etiquetas
-Probando suerte con BRkNN


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
10a,0.044,0.439,0.020,0.414,1.034,1.426,2.465,2.823,0.327,2.643,2.201,0.069,0.233,2.362,1.264,2.485e+00,0.138,3.476,0.156,0.965,1.963e-02,1.532,2.088,0.307,0.705,3.453,1.059,0.599,0.539,0.382,2.819,0.218,0.481,1.523,0.059,9.591e-01,0.049,0.003,0.112,0.519,1.411,1.766,0.175,1.068,0.060,0.033,0.940,0.059,2.968e-02,0.135,1.511e+00,0.355,0.597,0.261,2.000,0.169,0.072,0.154,0.025,2.105,0.071,0.959,0.048,0.084,1.827,0.285,1.824,0.994,0.758,2.499,0.016,1.583,0.114,0.665,1.614,2.056,0.588,2.719,0.166,0.412,0.335,0.173,2.205e+00,0.989,0.008,1.864,2.390,0.550,0.058,1.085,0.027,0.568,0.016,0.147,2.970,0.817,0.157,2.743,0.054,0.032,0.160,0.648,0.189,0.545,1.396,1.269,1.151,1.861,1.318,0.639,0.227,0.161,0.162,0.774,0.132,1.294e+00,2.766,0.863,0.246,0.187,0.270,2.119,1.859,0.266,0.018,0.449,0.491,0.478,0.892,0.017,0.044,0.189,1.413,2.244,2.243,2.651e-01,1.205,0.146,2.402e+00,0.045,1.077,0.008,1.035e+00,0.553,1.247,1.216,0.550,0.327,0.389,1.479e+00,0.062,1.555,1.117,1.642,0.078,1.938,0.040,0.353,0.065,1.058,0.103,0.343,0.952,0.218,0.310,1.292,0.817,0.476,0.477,0.981,0.400,0.151,0.104,0.263,1.970,0.577,0.323,2.982e+00,3.475,0.278,0.036,0.066,0.427,0.800,2.057,1.297e+00,0.261,0.349,0.039,2.457e+00,0.910,0.149,0.792,2.004,0.707,1.312,1.119,1.902,0.240,0.073,0.505,0.176,1.228,2.026,1.142,0.851,1.425e+00,1.852,1.376,1.175,1.280,0.190,1.468,0.164,0.202,1.943,1.365,0.581,2.720,0.387,1.248,1.511,0.462,1.551,0.812,0.221,0.776,0.220,2.038,2.637,0.191,0.037,0.343,1.885,1.736,1.205,0.036,1.001,0.164,1.879,1.058,2.748,1.315,1.096,0.403,0.299,0.202,1.698,0.169,2.177,1.335,0.081,0.118,2.028,1.964,2.347,0.548,0.624,0.452,0.156,0.340,1.450,1.753,0.734,1.162e+00,1.534,0.176,0.032,0.005,0.552,0.163,0.341,2.267,1.636,0.837,0.470,0.065,0.875,1.827e+00,2.775,5.550e-01,1.545,0.252,0.177,1.275,0.101,1.557,0.023,0.720,0.077,1.444,0.218,0.088,1.525,0.360,1.331e+00,1.847,1.608,1.402,0.206,0.066,0.232,0.855,0.082,1.234,2.099,1.492,2.073,1.317,1.054,1.389,0.147,0.652,1.212,1.591,0.383,0.499,0.104,0.548,0.103,2.673,0.963,1.931,0.494,1.509,0.828,6.295e-01,0.644,1.394,0.2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
10d,1.590,2.419,1.239,0.503,0.178,9.508e-02,0.895,0.366,1.248,0.113,2.620e-01,0.877,0.376,0.132,0.389,0.267,0.371,0.315,1.090,1.322,6.208e-04,1.938,5.718e-02,1.445,1.098e-01,0.205,0.223,1.801,1.736,1.136,0.203,1.240,0.494,0.286,1.181,1.488,1.629,0.146,0.024,1.183,0.059,0.154,0.084,0.077,0.576,1.098,1.701,0.870,0.003,1.166,0.341,1.786,1.736,1.138,0.140,0.005,0.725,0.039,0.008,0.386,1.577,0.676,0.682,0.003,0.391,1.089,0.484,0.776,0.272,0.439,0.450,1.329,1.339,1.121,0.221,7.406e-01,1.240,0.243,1.086,0.909,1.392,0.751,0.185,2.537,2.182,0.053,1.132e+00,1.420,1.354,1.092,2.030e-01,0.263,1.204,2.317,0.385,2.283,0.690,0.215,0.834,0.488,1.422,0.244,1.246,2.322,3.476e-01,0.184,2.051,2.193,0.555,0.382,0.078,0.832,0.861,0.917,1.509,0.084,0.258,0.000,0.779,0.004,1.959,1.485,0.075,0.925,1.177,1.873,1.264,2.045,1.370,0.469,0.011,1.166,1.708,0.063,1.972,0.209,0.510,0.228,0.814,1.571,0.117,0.253,0.120,1.650,8.428e-01,0.232,0.475,1.457,0.185,0.711,1.867,0.549,0.879,2.614e-01,1.337,3.870e-01,0.840,1.363,0.646,2.146e+00,2.291,1.822,0.583,1.119,1.156,0.386,0.725,0.759,0.316,1.898,0.814,0.642,0.163,0.914,1.030,1.530,1.468,3.568e-02,1.083e-01,1.104,0.005,0.020,1.368,0.026,1.405,0.136,0.260,1.963,0.039,0.091,0.807,0.393,0.491,0.233,0.964,0.870,1.677,0.389,2.334,0.117,1.104,1.597,0.971,7.632e-02,0.739,1.300,4.464e-02,0.689,2.792,1.352,0.897,2.059,0.755,1.807,0.489,1.250,1.951,1.677,0.111,1.674,0.228,0.161,1.742,1.784,0.092,1.369,1.911,1.476,1.939,0.103,0.002,1.242,0.079,3.324e-01,1.106e-01,0.588,1.335,0.995,0.731,0.110,1.833,0.134,1.273,0.367,1.313,0.857,0.135,0.653,0.322,1.113,1.181,1.147,1.124,0.484,0.123,0.408,1.922,2.828,1.435,0.312,1.159,0.124,0.534,3.631e-01,0.076,0.394,0.029,0.022,0.491,1.421,2.304,1.141,0.154,8.915e-02,2.271,0.247,0.387,1.085,0.518,0.049,0.206,0.528,1.983,0.081,0.146,0.010,0.135,1.001,0.280,1.234,2.442e-01,1.275,0.069,1.573,0.855,0.046,0.208,0.478,0.472,1.035e+00,0.367,1.613e-01,0.718,0.824,0.672,0.563,0.073,0.097,0.302,0.556,1.126,0.946,1.708,0.285,0.336,2.412,1.086,1.293,1.232,1.254,0.073,1.369,0.403,1.326,0.674,5.689e-01,0

,horizontal,panel,vertical,bar,enclosing,line,filling,ornament,cross,metopal,hatched,triangle,solid,double,hook,concentric,dotted,cross-hatched,dot,circle,outline,lozenge,meander,chain,swastika,floor
10a,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10b,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10c,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
10e,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
11a,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
11b,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11c,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11e,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11f,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11h,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


,horizontal,panel,vertical,bar,enclosing,line,filling,ornament,cross,metopal,hatched,triangle,solid,double,hook,concentric,dotted,cross-hatched,dot,circle,outline,lozenge,meander,chain,swastika,floor
10d,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
11d,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12b,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12f,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13b,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13h,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14d,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
14k,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15c,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16c,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
10a,0.044,0.439,0.020,0.414,1.034,1.426,2.465,2.823,0.327,2.643,2.201,0.069,0.233,2.362,1.264,2.485e+00,0.138,3.476,0.156,0.965,1.963e-02,1.532,2.088,0.307,0.705,3.453,1.059,0.599,0.539,0.382,2.819,0.218,0.481,1.523,0.059,9.591e-01,0.049,0.003,0.112,0.519,1.411,1.766,0.175,1.068,0.060,0.033,0.940,0.059,2.968e-02,0.135,1.511e+00,0.355,0.597,0.261,2.000,0.169,0.072,0.154,0.025,2.105,0.071,0.959,0.048,0.084,1.827,0.285,1.824,0.994,0.758,2.499,0.016,1.583,0.114,0.665,1.614,2.056,0.588,2.719,0.166,0.412,0.335,0.173,2.205e+00,0.989,0.008,1.864,2.390,0.550,0.058,1.085,0.027,0.568,0.016,0.147,2.970,0.817,0.157,2.743,0.054,0.032,0.160,0.648,0.189,0.545,1.396,1.269,1.151,1.861,1.318,0.639,0.227,0.161,0.162,0.774,0.132,1.294e+00,2.766,0.863,0.246,0.187,0.270,2.119,1.859,0.266,0.018,0.449,0.491,0.478,0.892,0.017,0.044,0.189,1.413,2.244,2.243,2.651e-01,1.205,0.146,2.402e+00,0.045,1.077,0.008,1.035e+00,0.553,1.247,1.216,0.550,0.327,0.389,1.479e+00,0.062,1.555,1.117,1.642,0.078,1.938,0.040,0.353,0.065,1.058,0.103,0.343,0.952,0.218,0.310,1.292,0.817,0.476,0.477,0.981,0.400,0.151,0.104,0.263,1.970,0.577,0.323,2.982e+00,3.475,0.278,0.036,0.066,0.427,0.800,2.057,1.297e+00,0.261,0.349,0.039,2.457e+00,0.910,0.149,0.792,2.004,0.707,1.312,1.119,1.902,0.240,0.073,0.505,0.176,1.228,2.026,1.142,0.851,1.425e+00,1.852,1.376,1.175,1.280,0.190,1.468,0.164,0.202,1.943,1.365,0.581,2.720,0.387,1.248,1.511,0.462,1.551,0.812,0.221,0.776,0.220,2.038,2.637,0.191,0.037,0.343,1.885,1.736,1.205,0.036,1.001,0.164,1.879,1.058,2.748,1.315,1.096,0.403,0.299,0.202,1.698,0.169,2.177,1.335,0.081,0.118,2.028,1.964,2.347,0.548,0.624,0.452,0.156,0.340,1.450,1.753,0.734,1.162e+00,1.534,0.176,0.032,0.005,0.552,0.163,0.341,2.267,1.636,0.837,0.470,0.065,0.875,1.827e+00,2.775,5.550e-01,1.545,0.252,0.177,1.275,0.101,1.557,0.023,0.720,0.077,1.444,0.218,0.088,1.525,0.360,1.331e+00,1.847,1.608,1.402,0.206,0.066,0.232,0.855,0.082,1.234,2.099,1.492,2.073,1.317,1.054,1.389,0.147,0.652,1.212,1.591,0.383,0.499,0.104,0.548,0.103,2.673,0.963,1.931,0.494,1.509,0.828,6.295e-01,0.644,1.394,0.2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
10d,1.590,2.419,1.239,0.503,0.178,9.508e-02,0.895,0.366,1.248,0.113,2.620e-01,0.877,0.376,0.132,0.389,0.267,0.371,0.315,1.090,1.322,6.208e-04,1.938,5.718e-02,1.445,1.098e-01,0.205,0.223,1.801,1.736,1.136,0.203,1.240,0.494,0.286,1.181,1.488,1.629,0.146,0.024,1.183,0.059,0.154,0.084,0.077,0.576,1.098,1.701,0.870,0.003,1.166,0.341,1.786,1.736,1.138,0.140,0.005,0.725,0.039,0.008,0.386,1.577,0.676,0.682,0.003,0.391,1.089,0.484,0.776,0.272,0.439,0.450,1.329,1.339,1.121,0.221,7.406e-01,1.240,0.243,1.086,0.909,1.392,0.751,0.185,2.537,2.182,0.053,1.132e+00,1.420,1.354,1.092,2.030e-01,0.263,1.204,2.317,0.385,2.283,0.690,0.215,0.834,0.488,1.422,0.244,1.246,2.322,3.476e-01,0.184,2.051,2.193,0.555,0.382,0.078,0.832,0.861,0.917,1.509,0.084,0.258,0.000,0.779,0.004,1.959,1.485,0.075,0.925,1.177,1.873,1.264,2.045,1.370,0.469,0.011,1.166,1.708,0.063,1.972,0.209,0.510,0.228,0.814,1.571,0.117,0.253,0.120,1.650,8.428e-01,0.232,0.475,1.457,0.185,0.711,1.867,0.549,0.879,2.614e-01,1.337,3.870e-01,0.840,1.363,0.646,2.146e+00,2.291,1.822,0.583,1.119,1.156,0.386,0.725,0.759,0.316,1.898,0.814,0.642,0.163,0.914,1.030,1.530,1.468,3.568e-02,1.083e-01,1.104,0.005,0.020,1.368,0.026,1.405,0.136,0.260,1.963,0.039,0.091,0.807,0.393,0.491,0.233,0.964,0.870,1.677,0.389,2.334,0.117,1.104,1.597,0.971,7.632e-02,0.739,1.300,4.464e-02,0.689,2.792,1.352,0.897,2.059,0.755,1.807,0.489,1.250,1.951,1.677,0.111,1.674,0.228,0.161,1.742,1.784,0.092,1.369,1.911,1.476,1.939,0.103,0.002,1.242,0.079,3.324e-01,1.106e-01,0.588,1.335,0.995,0.731,0.110,1.833,0.134,1.273,0.367,1.313,0.857,0.135,0.653,0.322,1.113,1.181,1.147,1.124,0.484,0.123,0.408,1.922,2.828,1.435,0.312,1.159,0.124,0.534,3.631e-01,0.076,0.394,0.029,0.022,0.491,1.421,2.304,1.141,0.154,8.915e-02,2.271,0.247,0.387,1.085,0.518,0.049,0.206,0.528,1.983,0.081,0.146,0.010,0.135,1.001,0.280,1.234,2.442e-01,1.275,0.069,1.573,0.855,0.046,0.208,0.478,0.472,1.035e+00,0.367,1.613e-01,0.718,0.824,0.672,0.563,0.073,0.097,0.302,0.556,1.126,0.946,1.708,0.285,0.336,2.412,1.086,1.293,1.232,1.254,0.073,1.369,0.403,1.326,0.674,5.689e-01,0

,horizontal,panel,vertical,bar,enclosing,line,filling,ornament,cross,metopal,hatched,triangle,solid,double,hook,concentric,dotted,cross-hatched,dot,circle,outline,lozenge,meander,chain,swastika,floor
10a,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10b,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10c,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
10e,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
11a,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
11b,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11c,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11e,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11f,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11h,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


,horizontal,panel,vertical,bar,enclosing,line,filling,ornament,cross,metopal,hatched,triangle,solid,double,hook,concentric,dotted,cross-hatched,dot,circle,outline,lozenge,meander,chain,swastika,floor
10d,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
11d,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12b,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12f,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13b,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13h,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14d,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
14k,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15c,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16c,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


---AVG Micro F1: 0.071
---AVG Micro recall: 0.0611
---AVG Micro precision: 0.0847
---AVG Accuracy: 0.0
---AVG Hamming Loss: 0.2076
---AVG Exact Match Ratio: 0.0
---AVG Hamming Score: 0.0346
---AVG 1-Match Ratio: 0.1753
---AVG 2-Match Ratio: 0.0206
---AVG 3-Match Ratio: 0.0103
---AVG 4-Match Ratio: 0.0
---AVG 5-Match Ratio: 0.0



In [6]:
Y_train.shape

(504, 26)